In [4]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown,display,update_display
from scraper import fetch_website_links,fetch_website_contents
from openai import OpenAI

In [5]:
load_dotenv(override=True)
api_key=os.getenv("OPENAI_API_KEY")

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [6]:
links=fetch_website_links("https://www.smxtech.com")
links

['/about-us/news/smx-announces-acquisition-of-cbeyondata/',
 'https://www.smxtech.com/',
 '#',
 '#',
 'https://www.smxtech.com/solutions/cybersecurity/',
 'https://www.smxtech.com/solutions/data-analytics/',
 'https://www.smxtech.com/solutions/digital-transformation-cloud-automation/',
 'https://www.smxtech.com/solutions/enterprise-applications/',
 'https://www.smxtech.com/solutions/mission-operations-and-intelligence/',
 'https://www.smxtech.com/solutions/modeling-simulation-training/',
 '#',
 'https://www.smxtech.com/solutions/smx-elevate-overview/',
 'https://www.smxtech.com/solutions/smx-elevate-fast-track/',
 'https://www.smxtech.com/solutions/smx-elevate-fast-track/isvs/',
 'https://www.smxtech.com/solutions/smx-elevate-fast-track/government/',
 'https://www.smxtech.com/solutions/smx-elevate-fm/',
 'https://www.smxtech.com/solutions/smx-elevate-isr/',
 'https://www.smxtech.com/solutions/smx-elevate-netedge/',
 'https://www.smxtech.com/solutions/smx-elevate-sensing/',
 'https://ww

In [7]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [9]:
print(get_links_user_prompt("https://www.smxtech.com"))


Here is the list of links on the website https://www.smxtech.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

/about-us/news/smx-announces-acquisition-of-cbeyondata/
https://www.smxtech.com/
#
#
https://www.smxtech.com/solutions/cybersecurity/
https://www.smxtech.com/solutions/data-analytics/
https://www.smxtech.com/solutions/digital-transformation-cloud-automation/
https://www.smxtech.com/solutions/enterprise-applications/
https://www.smxtech.com/solutions/mission-operations-and-intelligence/
https://www.smxtech.com/solutions/modeling-simulation-training/
#
https://www.smxtech.com/solutions/smx-elevate-overview/
https://www.smxtech.com/solutions/smx-elevate-fast-track/
https://www.smxtech.com/solutions/smx-elevate-fast-track/isvs/
https://www.smxtech.com/solutions/smx-elevate-fast-track/government/
h

In [10]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system","content":link_system_prompt},
            {"role":"user","content":get_links_user_prompt(url)}
        ],
        response_format={"type":"json_object"}
    )
    result = response.choices[0].message.content
    links=json.loads(result)
    return links

In [11]:
select_relevant_links("https://www.smxtech.com")

{'links': [{'type': 'about page', 'url': 'https://www.smxtech.com/about-us/'},
  {'type': 'about page',
   'url': 'https://www.smxtech.com/about-us/awards-recognition/'},
  {'type': 'about page', 'url': 'https://www.smxtech.com/about-us/clients/'},
  {'type': 'about page', 'url': 'https://www.smxtech.com/about-us/partners/'},
  {'type': 'about page', 'url': 'https://www.smxtech.com/about-us/leaders/'},
  {'type': 'about page',
   'url': 'https://www.smxtech.com/about-us/quality-business-certifications/'},
  {'type': 'doing business information',
   'url': 'https://www.smxtech.com/doing-business/'},
  {'type': 'contracts page',
   'url': 'https://www.smxtech.com/contracting-vehicles/'},
  {'type': 'news page',
   'url': 'https://www.smxtech.com/about-us/news/smx-announces-acquisition-of-cbeyondata/'},
  {'type': 'news page',
   'url': 'https://www.smxtech.com/about-us/news/energage-names-smx-a-winner-of-the-2025-top-workplaces-usa/'},
  {'type': 'news page',
   'url': 'https://www.smxte

In [12]:
### Second step -- Brochure

In [13]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
fetch_page_and_all_relevant_links("https://www.smxtech.com")

'## Landing Page:\n\nSMX - Harness Technology to Realize Your Digital Future\n\nSMX announces the acquisition of cBEYONData, enhancing our digital transformation capabilities and amplifying our ability to improve the business of government.\nSolutions\nAll Solutions\nCybersecurity\nData & Analytics\nDigital Transformation\nEnterprise Applications\nMission Operations & Intelligence\nModeling, Simulation & Training\nAccelerators\nSMX Elevate\nElevate Fast Track\nISVs\nGovernment\nElevate FM\nElevate ISR\nElevate NetEdge\nElevate Sensing\nGenerative AI\nInsights\nNews\nCustomer Spotlights\nResources\nEvents\nCareers\nOpen Positions\nAll Career Opportunities\nDMV Cleared Roles\nHawaii Careers\nSouthern Maryland Careers\nBenefits\nSkillBridge Fellowship Program\nAbout\nAbout Us\nAwards & Recognition\nClients\nContracting Vehicles\nPartners\nLeadership & Governance\nQuality & Business Certifications\nDoing Business\nContact\nSelect Page\nOur QSMO-approved Core FS solution enhances federal fi

In [15]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("SMX Technologies", "https://www.smxtech.com")

'\nYou are looking at a company called: SMX Technologies\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nSMX - Harness Technology to Realize Your Digital Future\n\nSMX announces the acquisition of cBEYONData, enhancing our digital transformation capabilities and amplifying our ability to improve the business of government.\nSolutions\nAll Solutions\nCybersecurity\nData & Analytics\nDigital Transformation\nEnterprise Applications\nMission Operations & Intelligence\nModeling, Simulation & Training\nAccelerators\nSMX Elevate\nElevate Fast Track\nISVs\nGovernment\nElevate FM\nElevate ISR\nElevate NetEdge\nElevate Sensing\nGenerative AI\nInsights\nNews\nCustomer Spotlights\nResources\nEvents\nCareers\nOpen Positions\nAll Career Opportunities\nDMV Cleared Roles\nHawaii Careers\nSouthern Maryland Careers\nBenefits\nSkillBridge Fellowship Program\nAbout\

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("SMX Technologies", "https://www.smxtech.com")

# SMX Technologies Brochure

---

## Harness Technology to Realize Your Digital Future

SMX Technologies is a leading technology company dedicated to driving digital transformation primarily for government clients. With advanced technology woven into its DNA, SMX empowers organizations to accelerate mission impact securely and at scale.

---

## Our Solutions

- **Cybersecurity:** Protecting critical infrastructures and sensitive data with cutting-edge security solutions.
- **Data & Analytics:** Turning data into actionable insights for smarter decision-making.
- **Digital Transformation:** Automating and optimizing business processes to deliver greater outcomes.
- **Enterprise Applications:** Enhancing operational efficiency through customized software solutions.
- **Mission Operations & Intelligence:** Supporting government missions with innovative ISR (Intelligence, Surveillance, Reconnaissance) platforms.
- **Modeling, Simulation & Training:** Delivering immersive, effective training and simulation experiences.
- **Accelerators:** Including SMX Elevate, Elevate Fast Track, ISVs, Elevate FM, Elevate ISR, Elevate NetEdge, Elevate Sensing, and Generative AI to optimize various mission-critical workflows.

---

## Strategic Growth & Innovation

- **Acquisition of cBEYONData:** This recent acquisition expands SMX’s digital transformation capabilities and strengthens its ability to support government agencies.
- **QSMO-approved Core FS Solution:** Enhances federal financial management processes to maximize mission impact.
- **SMX Elevate AI Labs℠:** A secure AI experimentation environment designed to protect proprietary information and enable safe innovation.
- **Commercial ISR Platform:** Accelerating insights with better outcomes, value, and agility for intelligence operations.

---

## Our Customers

SMX serves a broad range of government clients, particularly focused on federal agencies. Our technologies and solutions amplify government capabilities in financial management, intelligence, mission operations, and cybersecurity.

---

## Company Culture & Recognition

- **Top Workplace:** SMX has earned the prestigious Energage Top Workplaces USA award for three consecutive years, reflecting a supportive, innovative, and employee-centered culture.
- **SkillBridge Fellowship Program:** We invest in developing talent by offering fellowship programs that provide real-world experience.

---

## Career Opportunities

SMX is actively hiring across various locations including:
- DMV Area (Washington D.C., Maryland, Virginia) with cleared roles
- Hawaii
- Southern Maryland

**Benefits:** Competitive compensation, professional development, and a collaborative working environment focused on innovation and mission success.

Explore open positions and join a company that values your expertise and growth.

---

## Leadership & Governance

SMX operates with strong leadership committed to ethical business practices and high-quality delivery as evidenced by multiple business certifications and contracting vehicle affiliations.

---

## Contact & Further Information

Discover how SMX Technologies can help you harness technology for your digital future. Visit [smxtech.com](https://www.smxtech.com) for more details.

---

**SMX Technologies**  
*Accelerate mission and business impact, securely and at scale.*  
Shared Vision. Outcome Assured.